#Import OpenAI SDK

In [9]:
# Install the openai-agents library
!pip install -Uq openai-agents

#Import Asyncio to run asynchronus functions

In [ ]:
# Import the nest_asyncio library
import nest_asyncio
# Apply nest_asyncio to allow asyncio to run in a nested environment (like Colab)
nest_asyncio.apply()

#Set Gemini API key as Default API reference instaed of Open API key and Setting Up Configurations

In [6]:
# Import necessary libraries
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata
from pydantic import BaseModel

from agents import (
    Agent,
    Runner,
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled,
)

# Get the Gemini API key from Colab user data secrets
gemini_api_key = userdata.get('GEMINI_API_KEY')

# Disable tracing for the agents library
set_tracing_disabled(True)
# Set the default OpenAI API to chat completions
set_default_openai_api("chat_completions")

# Create an AsyncOpenAI client with the Gemini API key and base URL
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
# Set the default OpenAI client to the external client
set_default_openai_client(external_client)

# Create an OpenAIChatCompletionsModel instance using the Gemini 2.0 Flash model
model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client= external_client
)

#Hosted Tools: WebSearch Tool, FileSearch Tool, Computer Tool

In [ ]:
# Import necessary components from the agents library
from agents import Agent, FileSearchTool, Runner, WebSearchTool # hosted tools require OPEN API key so it won't work with Gemini

# Create an Agent named "Assistant" with WebSearchTool and FileSearchTool
agent = Agent(
    name="Assistant",
    tools=[
        WebSearchTool(), # Tool for web searching
        FileSearchTool( # Tool for file searching
            max_num_results=3, # Maximum number of results to return
            # vector_store_ids=["VECTOR_STORE_ID"], # Example of how to specify vector store IDs
            vector_store_ids=["vs_6813268d82a081919782a0990f3a68f9"], # Specific vector store ID
        ),
    ],
)

In [ ]:
# Run the agent synchronously with the input "Show Muhammad Qasim current organization and job title"
result =  Runner.run_sync(agent, "Show Muhammad Qasim current organization and job title")
# Print the final output from the agent's run
print(result.final_output)

#Streaming
##1. Streaming Text code

In [7]:
# Import necessary libraries
import asyncio

from openai.types.responses import ResponseTextDeltaEvent

from agents import Agent, Runner


# Define an asynchronous main function
async def main():
    # Create an Agent named "Joker" with instructions and the specified model
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model=model
    )

    # Run the agent with streaming enabled and the input "Please tell me 5 jokes."
    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    # Iterate through the streamed events
    async for event in result.stream_events():
        # Check if the event is a raw response event and contains text delta
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            # Print the text delta without a newline, flushing the output immediately
            print(event.data.delta, end="", flush=True)


# Run the main asynchronous function
asyncio.run(main())

Alright, here are 5 jokes for you:

1.  Why don't scientists trust atoms?
    Because they make up everything!

2.  What do you call a lazy kangaroo?
    Pouch potato!

3.  Why did the bicycle fall over?
    Because it was two tired!

4.  Why did the scarecrow win an award?
    Because he was outstanding in his field!

5.  What musical instrument is found in the bathroom?
    A tuba toothpaste.


##2. Stream item code

In [8]:
# Import necessary libraries
import asyncio
import random

from agents import Agent, ItemHelpers, Runner, function_tool


# Define a function tool that returns a random integer between 1 and 10
@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


# Define an asynchronous main function
async def main():
    # Create an Agent named "Joker" with instructions, the defined tool, and the specified model
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=model
    )

    # Run the agent with streaming enabled and the input "Hello"
    result = Runner.run_streamed(
        agent,
        input="Hello",

    )
    print("=== Run starting ===")
    # Iterate through the streamed events
    async for event in result.stream_events():
        # Ignore raw response event deltas
        if event.type == "raw_response_event":
            continue
        # Print a message when the agent is updated
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # Handle different types of run item stream events
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types


# Run the main asynchronous function
asyncio.run(main())

print("=== Run complete ===")

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 2
-- Message output:
 I am programmed to tell 2 jokes.

Why don't scientists trust atoms?

Because they make up everything!

 параллельно

Why did the scarecrow win an award?

Because he was outstanding in his field!

=== Run complete ===


#Context
##1. Local Context

In [14]:
# Import necessary libraries
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

# Define a dataclass to hold user information
@dataclass
class UserInfo1():
    name: str
    uid: int
    age: int
    location: str = "Pakistan" # Default location is Pakistan


# Define a function tool to fetch user's age
@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo1]) -> str:
    '''Returns the name and age of the user.'''

    print("[-> Tool]",wrapper,"\n\n")
    return f"User {wrapper.context.name} is {wrapper.context.age} years old"

# Define a function tool to fetch user's location
@function_tool
async def fetch_user_location(wrapper: RunContextWrapper[UserInfo1]) -> str:
    '''Returns the location of the user.'''
    return f"User {wrapper.context.name} is from {wrapper.context.location}"

# Define an asynchronous main function
async def main():
    # Create an instance of UserInfo1
    user_info = UserInfo1(name="Muhammad Qasim", uid=123,age=30)

    # Create an Agent with UserInfo1 context, tools, and the specified model
    agent = Agent[UserInfo1](
        name="Assistant",
        tools=[fetch_user_age,fetch_user_location],
        model=model
    )

    # Run the agent with the input and user_info as context
    result = await Runner.run(
        starting_agent=agent,
        input="What is the age of the user? current location of his/her?",
        context=user_info,
    )

    # Print the final output from the agent's run
    print(result.final_output)
    # Expected output: The user John is 47 years old.

# Run the main asynchronous function if the script is executed directly
if __name__ == "__main__":
    asyncio.run(main())

[-> Tool] ToolContext(context=UserInfo1(name='Muhammad Qasim', uid=123, age=30, location='Pakistan'), usage=Usage(requests=1, input_tokens=41, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=10, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=51), tool_call_id='') 


The user Muhammad Qasim is 30 years old and is from Pakistan.



##2. Dynamic Instructions with context

In [15]:
# Import necessary libraries
import asyncio
import random
from typing import Literal

from agents import Agent, RunContextWrapper, Runner

# Define a class for custom context with a style attribute
class CustomContext:
    def __init__(self, style: Literal["haiku", "pirate", "robot"]):
        self.style = style

# Define a function to generate dynamic instructions based on the context style
def custom_instructions(
    run_context: RunContextWrapper[CustomContext], agent: Agent[CustomContext]
) -> str:
    context = run_context.context
    if context.style == "haiku":
        return "Only respond in haikus."
    elif context.style == "pirate":
        return "Respond as a pirate."
    else:
        return "Respond as a robot and say 'beep boop' a lot."

# Create an Agent with dynamic instructions and the specified model
agent = Agent(
    name="Chat agent",
    instructions=custom_instructions,
    model=model
)

# Define an asynchronous main function
async def main():
    # Randomly choose a style for the custom context
    choice: Literal["haiku", "pirate", "robot"] = random.choice(["haiku", "pirate", "robot"])
    # Create an instance of CustomContext with the chosen style
    context = CustomContext(style=choice)
    print(f"Using style: {choice}\n")

    user_message = "Tell me a joke."
    print(f"User: {user_message}")
    # Run the agent with the user message and custom context
    result = await Runner.run(agent, user_message, context=context)

    # Print the final output from the agent's run
    print(f"Assistant: {result.final_output}")

# Run the main asynchronous function if the script is executed directly
if __name__ == "__main__":
    asyncio.run(main())

Using style: pirate

User: Tell me a joke.
Assistant: Aye, I've got a joke for ye, sure as the sea is blue!

Why don't pirates take a shower?

... Because they just wash up on shore! 

Har har har! Did ye find that funny, me hearty? Or should I walk the plank and find another?



##3. Agent/LLM context

In [17]:
# Import necessary libraries
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

# Define a dataclass to hold user information
@dataclass
class UserInfo:
    name: str
    uid: int

# Define a function tool to greet the user with a specialized message
@function_tool
async def greet_user(context: RunContextWrapper[UserInfo], greeting: str) -> str:
  """Greets the User with their name.
  Args:
    greeting: A specialized greeting message for user
  """
  name = context.context.name
  return f"Hello {name}, {greeting}"

# Define an asynchronous main function
async def main():
    # Create an instance of UserInfo
    user_info = UserInfo(name="Muhammad Qasim", uid=123)

    # Create an Agent with UserInfo context, the greet_user tool, and the specified model
    agent = Agent[UserInfo](
        name="Assistant",
        tools=[greet_user],
        model=model,
        # Dynamic function context passed in instructions/system prompt/developer prompt
        instructions="Always greet the user using <function_call>greet_user</function_call> and welcome them to Panaversity, then answer the user's prompt"
    )

    # Run the agent with the input and user_info as context
    result = await Runner.run(
        starting_agent=agent,
        input="who is the founder of pakistan",
        context=user_info,
    )

    # Print the final output from the agent's run
    print(result.final_output)

# Run the main asynchronous function
asyncio.run(main())

Hello Muhammad Qasim, Hello! Welcome to Panaversity!

Muhammad Ali Jinnah is considered the founder of Pakistan.

